<a href="https://colab.research.google.com/github/KauaHenSilva/python_nn_breast_cancer/blob/main/breast_cancer_cruzada.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Recuperar arquivos

In [34]:
import os
tipo_de_recuperacao = "Ambos" #@param ["Download", "Drive", "Ambos"]
def repostirio_para_download():
  if os.path.exists('python_download_file_drive'):
    !cd python_download_file_drive && git pull
  else:
    !git clone https://github.com/KauaHenSilva/python_download_file_drive
  %pip install --upgrade --quiet -r python_download_file_drive/requirements.txt

## Recuperação por download

In [35]:
if tipo_de_recuperacao in ["Download", "Ambos"]:
  from google.colab import auth
  from googleapiclient.discovery import build
  auth.authenticate_user()
  repostirio_para_download()
  !python "python_download_file_drive/main.py" "https://drive.google.com/drive/folders/183Mj7Uk4jLSbP46-UISygf3gmdp8Y-Pt?usp=drive_link" arquivos

Already up to date.
Baixando arquivos: 100% 2/2 [00:00<00:00, 38657.18arquivo/s]


## Recuperação por drive

In [36]:
# @markdown #### Caso escolheu o tipo de recuperação drive/ambos e não tenha os arquivos, faça um atalho desse drive:
# @markdown > https://drive.google.com/drive/folders/183Mj7Uk4jLSbP46-UISygf3gmdp8Y-Pt?usp=drive_link
path_atalho_drive = "/content/drive/MyDrive/Projetos Do GitHub/tmp_name" #@param {type:"string"}

In [37]:
if tipo_de_recuperacao in ["Drive", "Ambos"]:
  from google.colab import drive
  drive.mount('/content/drive')
  if not os.path.exists(path_atalho_drive):
    raise Exception("O caminho especificado não existe no drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Referenciar arquivo no drive ou download

In [38]:
def get_referencia_valida(path):
    path_download = os.path.join("/content/arquivos", path)
    path_drive = os.path.join(path_atalho_drive, path)

    bool_download = os.path.exists(path_download)
    bool_drive = os.path.exists(path_drive)

    if not bool_download and not bool_drive:
        raise Exception(f"Arquivo não encontrado em nenhum dos caminhos (Download ou Drive). Caminhos: {path_download}, {path_drive}")

    if not bool_drive and bool_download:
        print("[INFO] Arquivo encontrado no download, mas não no drive.")
        return path_download

    if bool_drive and not bool_download:
        print("[INFO] Arquivo encontrado no drive, mas não no download.")
        return path_drive

    return path_download


# breast cancer cruzada

In [39]:
!pip install tensorflow==2.16.1 scikit-learn==1.5.0  scikeras==0.13.0 --quiet

In [40]:
import scikeras
import pandas as pd
import tensorflow as tf
import sklearn

In [41]:
pd.__version__, tf.__version__, sklearn.__version__, scikeras.__version__

('2.2.2', '2.16.1', '1.5.0', '0.13.0')

In [42]:
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras import backend as k

In [43]:
path_entrada = get_referencia_valida("entradas_breast.csv")
path_saida = get_referencia_valida("saidas_breast.csv")

In [44]:
x = pd.read_csv(path_entrada)
y = pd.read_csv(path_saida)
x.shape, y.shape

((569, 30), (569, 1))

In [45]:
def criar_rede():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

def criar_rede_drop():
  k.clear_session()
  rede_neural = Sequential([
      tf.keras.layers.InputLayer(shape=(30,)),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=16, activation='relu', kernel_initializer='random_uniform'),
      tf.keras.layers.Dropout(rate = 0.2),
      tf.keras.layers.Dense(units=1, activation = 'sigmoid')])
  otimizador = tf.keras.optimizers.Adam(learning_rate = 0.001, clipvalue = 0.5)
  rede_neural.compile(optimizer = otimizador, loss = 'binary_crossentropy', metrics = ['binary_accuracy'])
  return rede_neural

In [46]:
rede_neural = KerasClassifier(model = criar_rede, epochs = 100, batch_size = 10)
rede_neural_drop = KerasClassifier(model = criar_rede_drop, epochs = 100, batch_size = 10)
rede_neural.get_metadata_routing(), rede_neural_drop.get_metadata_routing()

({'fit': {'sample_weight': None}, 'partial_fit': {'classes': None, 'sample_weight': None}, 'score': {'sample_weight': None}},
 {'fit': {'sample_weight': None}, 'partial_fit': {'classes': None, 'sample_weight': None}, 'score': {'sample_weight': None}})

In [47]:
resultado = cross_val_score(rede_neural, x, y, cv = 10,  scoring = 'accuracy')
resultado_drop = cross_val_score(rede_neural, x, y, cv = 10,  scoring = 'accuracy')

Epoch 1/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - binary_accuracy: 0.5824 - loss: 1.2223
Epoch 2/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.6198 - loss: 0.8082
Epoch 3/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7889 - loss: 0.4790
Epoch 4/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8302 - loss: 0.4145
Epoch 5/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8675 - loss: 0.4003
Epoch 6/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.7590 - loss: 0.6764
Epoch 7/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8234 - loss: 0.5309
Epoch 8/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8650 - loss: 0.3528
Epoch 9/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8460 - loss: 0.4156
Epoch 10/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8377 - loss: 0.4383
Epoch 11/100
52/52 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - binary_accuracy: 0.8521 -

In [48]:
resultado, resultado_drop

(array([0.80701754, 0.75438596, 0.85964912, 0.75438596, 0.66666667,
        0.84210526, 0.84210526, 0.80701754, 0.70175439, 0.875     ]),
 array([0.80701754, 0.8245614 , 0.84210526, 0.75438596, 0.70175439,
        0.89473684, 0.84210526, 0.89473684, 0.66666667, 0.85714286]))

In [49]:
resultado.mean(), resultado_drop.mean()

(0.7910087719298244, 0.8085213032581453)